In [1]:
!pip install vllm transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 29.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... \^C
Traceback (most recent call last):
  File "/Users/stefan/Desktop/Eurecom/Semester Project/venv/bin/pip", line 10, in <module>
    sys.exit(main())
             ~~~~^^
  File "/Users/stefan/Desktop/Eurecom/Semester Project/venv/lib/python3.13/site-packages/pip/_internal/cli/main.py", line 80, in main
    return command.main(cmd_args)
           ~~~~~~~~~~~~^^^^^^^^^^
  File "/Users/stefan/Desktop/Eurecom/Semester Project/venv/lib/python3.13/site-packages/pip/_internal/cli/base_command.py", line 156, in main
    with self.main_context():
         ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/contextlib.py", line 148, in __exit__
    next(self.gen)
    ~~~~^^^^^^^^^^
  File "/Users/stefan/Desktop/Eurecom/Semester Project/venv/lib/python3.13/site-packages/pip/_internal/cli/command_con

In [ ]:
!pip install litellm

In [ ]:
!nohup vllm serve Qwen/Qwen3-1.7B \
  --enable-auto-tool-choice \
  --tool-call-parser hermes \
  --reasoning-parser deepseek_r1 \
  > vllm.log &

nohup: redirecting stderr to stdout


In [ ]:
!tail -n 20 vllm.log # to watch last 20 rows og vllm.log, about 2/3 minutes to have the server ready -> Application startup complete

  File "/usr/local/lib/python3.12/dist-packages/vllm/entrypoints/openai/cli_args.py", line 263, in make_arg_parser
    parser = AsyncEngineArgs.add_cli_args(parser)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/engine/arg_utils.py", line 1714, in add_cli_args
    parser = EngineArgs.add_cli_args(parser)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/engine/arg_utils.py", line 919, in add_cli_args
    vllm_kwargs = get_kwargs(VllmConfig)
                  ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/engine/arg_utils.py", line 281, in get_kwargs
    return copy.deepcopy(_compute_kwargs(cls))
                         ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/engine/arg_utils.py", line 182, in _compute_kwargs
    default = field.default_factory()
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist

In [ ]:
import sqlite3
import pandas as pd

def query_db(sql: str):
    """
    Executes an SQL query and returns the result as JSON

    Args:
        sql: The SQL query to execute

    Returns:
        The result of the query as JSON
    """
    try:
        df = pd.read_sql_query(sql, conn)
        return df.to_dict(orient="records")
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# Dopo aver creato la tabella 'employees', aggiungi questo:
conn = sqlite3.connect("company.db") # Assicurati che conn sia attiva

# Modifica i dati 'employees' per includere 'dept_id'
data_employees = {
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Charlie"],
    "hire_date": ["2020-01-15", "2021-06-20", "2022-03-10"],
    "salary": [50000, 60000, 70000],
    "dept_id": [1, 2, 1] # Alice e Charlie sono nel Dip. 1, Bob nel 2
}
df_employees = pd.DataFrame(data_employees)
df_employees.to_sql("employees", conn, if_exists="replace", index=False)

# Crea dati 'departments'
data_departments = {
    "dept_id": [1, 2],
    "dept_name": ["Engineering", "Marketing"]
}
df_departments = pd.DataFrame(data_departments)
df_departments.to_sql("departments", conn, if_exists="replace", index=False)


In [ ]:
def get_function_by_name(name):
    if name == "query_db":
        return query_db

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "query_db",
            "description": "Executes a single SQL query on the database. Use this to explore the schema (e.g., PRAGMA table_info) or to get the final answer.",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql": {
                        "type": "string",
                        "description": 'The SQL query to execute on the database.',
                    }
                },
                "required": ["sql"],
            },
        },
    }
]

In [ ]:
USER_QUESTION = "Find the name of the highest-paid employee and also list their department's name."

SYSTEM_PROMPT = """You are an expert SQL assistant. Your task is to answer complex user questions by generating a final SQL query.

You do not know the database schema in advance.

You must follow a strict multi-step process:
1.  **Initial Analysis:** Think (in the reasoning/thinking trace) about what you need to discover.
2.  **Exploration (Loop):** You must first explore the database.
    * Use `SELECT name FROM sqlite_master WHERE type='table'` to find the tables.
    * Use `PRAGMA table_info(table_name)` to understand a table's columns.
    * Use `SELECT * FROM table_name LIMIT 3` to see sample data, if necessary.
    * Use the `query_db` tool for *EVERY SINGLE* exploratory query.
3.  **Intermediate Reasoning:** After each call to `query_db`, analyze the result and explain (in the reasoning trace) what you have learned and what your next exploratory step will be.
4.  **Final Construction:** Only when you are *absolutely sure* you have all the information (table names, column names, data types, keys for JOINs), build the final SQL query that answers the user's question.
5.  **Final Answer:** Execute the final query with `query_db` and provide the answer to the user in natural language, based on the result.

Do not attempt to answer the user's question with a single query before you have explored the schema."""

In [ ]:
from litellm import completion

model_name = "hosted_vllm/Qwen/Qwen3-1.7B"
MAX_TURNS = 6 # Imposta un limite per evitare loop infiniti

# Inizializza la cronologia della conversazione
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_QUESTION}
]

# Parametri comuni per 'completion'
completion_params = {
    "model": model_name,
    "temperature": 0.1, # Bassa temperatura per ragionamenti più logici
    "top_p": 0.8,
    "max_tokens": 1024, # Aumenta per trace più lunghe
    "repetition_penalty": 1.05,
    "extra_body": {
        "chat_template_kwargs": {
            "enable_thinking": True,
            "detailed_thinking": True
        }
    },
    "api_base": "http://127.0.0.1:8000/v1",
    "tools": TOOLS
}

print(f"🚀 Inizio Ragionamento per: '{USER_QUESTION}'\n")

for i in range(MAX_TURNS):
    print(f"\n--- 🔄 Turno {i+1} ---")

    # 1. Chiama il modello
    response = completion(messages=messages, **completion_params)
    response_message = response.choices[0].message

    # Aggiungi la risposta del modello (pensiero + tool_call) alla cronologia
    messages.append(response_message.model_dump())

    # Stampa la "traccia di ragionamento" (se il modello la fornisce)
    # Nota: la traccia di vLLM potrebbe essere nel log o in un campo non standard
    # Controlliamo 'content' se c'è pensiero prima del tool_call
    if response_message.content:
         print(f"🤔 Pensiero del Modello:\n{response_message.content}")

    # 2. Controlla se il modello ha chiamato uno strumento
    if tool_calls := response_message.get("tool_calls", None):
        print("🛠️ Chiamata Strumento Rilevata:")

        # 3. Esegui tutti i tool_calls
        for tool_call in tool_calls:
            call_id = tool_call["id"]
            if fn_call := tool_call.get("function"):
                fn_name = fn_call["name"]
                fn_args = json.loads(fn_call["arguments"])
                sql_query = fn_args.get("sql", "")

                print(f"   -> Esecuzione: {sql_query}")

                # Esegui la funzione
                fn_res = get_function_by_name(fn_name)(**fn_args)
                print(f"   <- Risultato DB: {fn_res}")

                # 4. Aggiungi il risultato dello strumento alla cronologia
                messages.append({
                    "role": "tool",
                    "content": json.dumps({
                        "query_eseguita": sql_query,
                        "risultato": fn_res
                    }),
                    "tool_call_id": call_id,
                })

    # 5. Controlla se il modello ha finito (ha dato una risposta finale senza chiamare strumenti)
    elif response_message.content:
        print(f"\n✅ Risposta Finale del Modello:\n{response_message.content}")
        break # Il ciclo è finito

    else:
        print("⚠️ Il modello non ha fornito né 'content' né 'tool_calls'. Interrompo.")
        break

if i == MAX_TURNS - 1:
    print("\n🚫 Limite turni raggiunto. Il modello non ha completato il ragionamento.")

# Chiudi la connessione al DB
conn.close()